# Assignment #6 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2024</br>

Name: Udoy Chowdhury
</br>
Date: 2/25/2024
<br>
<br>
**At this time in the semester:** <br>
--We have explored a dataset. <br>
--We have cleaned our dataset. <br>
--We created a Github account with a repository for this class and included a Metadata file about our data. <br>
--We introduced general SQL syntax, queries, and applications in R.<br>
<br>

Now we will start the process of uploading our dataset into a database. There are many different ways to upload your .csv data into a database (.db file). Databases can be created in many open source applications, MySQL workbench, and even some websites can load your .csv data into a database...for a small fee. Instead of using an application, we are going to first create our database for our dataset from scratch in R. On a much larger scale, data may be automatically uploaded to a database was once it is aquired.<br>

There are two R packages below we will use to create two separate databases for practice. One will be stored on our MySQL server using R package RMariaDB, and the other will be stored locally using R package SQLite. <br>
<br>
Follow the instructions below to complete the assignment. Answer any questions in markdown cell boxes. Be sure to comment all code.


### Creating our database from scratch to integrate with MySQL Workbench in Python.<br>

### Installs and Imports ###

In [2]:
pip install pandas sqlalchemy pymysql

     |████████████████████████████████| 44 kB 2.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Handling Data
import pandas as pd
# MySQL interaction
import mysql.connector
# Loading Data
from sqlalchemy import create_engine
# Create a DB file
import sqlite3

# Get rid of display restrictions
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


### Connect and Create Database ###

In [103]:
# Define our variables. We set these during our first class in our technology set up. 
password = 'chowdhury626'
username = 'root'
port = 3306
database_name = 'male_fifa_players'
host = 'localhost'

In [104]:
# Connect to the database
con = pymysql.connect(host=host, user=username, password=password, database=database_name)

In [15]:
# Create cursor
cursor = con.cursor()

# Create database
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Specify the database
cursor.execute(f"USE {database_name}")

0

### Load in Data ###

In [118]:
# Specify the path to your CSV file
csv_file_path = "/Users/udoychowdhury/Documents/DataScience/Soccer Data/male_players_cleaned.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Columns that need to be date
date_columns = ['dob', 'club_joined_date', 'club_contract_valid_until_year']  

# Convert columns to date then replace nulls
for col in date_columns:
    df[col] = pd.to_datetime(df[col]).dt.date.astype(str)
    df[col] = df[col].astype(str).replace('NaT', 'NULL')

# Handle Null Values
# Fill numeric with 0
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = df[col].fillna(0)

# Fill object and datetime with 'NULL'
for col in df.select_dtypes(include=['object', 'datetime64[ns]']).columns:
    df[col] = df[col].fillna('NULL')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (108) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [119]:
# Preview the dataframe
display(df)

KeyboardInterrupt: 

In [69]:
# Display data types
display(df.dtypes)

player_id                           int64
fifa_version                        int64
short_name                         object
long_name                          object
player_positions                   object
overall                             int64
potential                           int64
value_eur                           int64
value_dol                           int64
wage_eur                            int64
wage_dol                            int64
age                                 int64
dob                                object
height_cm                           int64
height_in                         float64
weight_kg                           int64
weight_lbs                        float64
club_team_id                        int64
club_name                          object
league_id                           int64
league_name                        object
league_level                        int64
club_position                      object
club_jersey_number                

In [28]:
# Preview the data
df.head()

,player_id,fifa_version,short_name,long_name,player_positions,overall,potential,value_eur,value_dol,wage_eur,wage_dol,age,dob,height_cm,height_in,weight_kg,weight_lbs,club_team_id,club_name,league_id,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,work_rate,body_type,release_clause_eur,release_clause_dol,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,24,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,181500000,199650000,230000,253000,24,1998-12-20,182,71.65,75,165.35,73,Paris Saint Germain,16,Ligue 1,1,LW,7,NaN,2018-07-01,1970-01-01,18,France,1335,LW,10,Right,4,5,High/Low,Unique,349400000,384340000,"#Speedster, #Dribbler, #Acrobat, #Clinical fin...","Quick Step +, Rapid, Flair, Trivela",97,90,80,92,36,78,78,94,73,86,84,93,80,69,71,92,97,97,93,93,82,90,88,88,77,83,64,38,93,83,84,88,26,34,32,13,5,7,11,6,NaN,90+3,90+3,90+3,91,91,91,91,91,89+3,89+3,89+3,89+3,81+3,81+3,81+3,89+3,68+3,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,24,E. Haaland,Erling Braut Haaland,ST,91,94,185000000,203500000,340000,374000,22,2000-07-21,195,76.77,94,207.23,10,Manchester City,13,Premier League,1,ST,9,NaN,2022-07-01,1970-01-01,36,Norway,1352,ST,9,Left,3,3,High/Medium,Unique,356100000,391710000,"#Aerial threat, #Distance shooter, #Strength, ...","Acrobatic +, Power Header, Quick Step",89,93,66,80,45,88,47,96,83,77,90,79,77,62,53,82,82,94,76,94,72,94,93,76,93,86,87,43,96,74,84,87,38,47,29,7,14,13,11,7,NaN,90+3,90+3,90+3,82,86,86,86,82,82+3,82+3,82+3,79+3,74+3,74+3,74+3,79+3,62+3,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,24,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,103000000,113300000,350000,385000,32,1991-06-28,181,71.26,75,165.35,10,Manchester City,13,Premier League,1,SUB,17,NaN,2015-08-30,1970-01-01,7,Belgium,1325,CAM,7,Right,5,4,High/Medium,Unique,190600000,209660000,"#Dribbler, #Playmaker, #Distance shooter, #Cro...","Pinged Pass +, Dead Ball, Incisive Pass, Long ...",72,88,94,87,65,78,95,85,55,94,83,86,92,83,94,92,72,72,74,92,78,92,72,88,74,92,75,66,88,95,83,88,66,70,53,15,13,5,10,13,NaN,83+3,83+3,83+3,87,88,88,88,87,89+2,89+2,89+2,88+3,90+1,90+1,90+1,88+3,79+3,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,24,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,41000000,45100000,23000,25300,36,1987-06-24,169,66.54,67,147.71,112893,Inter Miami,39,Major League Soccer,1,RF,10,NaN,2023-07-16,1970-01-01,52,Argentina,1369,RW,10,Left,4,4,Low/Low,Unique,61500000,67650000,"#Dribbler, #Playmaker, #FK Specialist, #Acroba...","Technical +, Finesse Shot, Dead Ball, Pinged P...",80,87,90,94,33,64,83,89,60,91,86,96,93,93,90,93,87,74,91,88,95,83,71,70,68,90,44,40,91,92,75,96,20,35,24,6,11,15,14,8,NaN,85+3,85+3,85+3,90,89,89,89,90,91-1,91-1,91-1,89+1,85+3,85+3,85+3,89+1,64+3,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,24,K. Benzema,Karim Benzema,"CF, ST",90,90,51000000,56100000,95000,104500,35,1987-12-19,185,72.83,81,178.57,607,Al Ittihad,350,Pro League,1,RS,9,NaN,2023-07-01,1970-01-01,18,France,0,NaN,0,Ri

In [30]:
# Check the column names
display(df.columns)

Index(['player_id', 'fifa_version', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur', 'value_dol',
       'wage_eur',
       ...
       'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],
      dtype='object', length=109)

### Create Tables Within Database ###

In [82]:
# Seperate df into 3 different df

# Creating player_info df
player_info_columns = [
    'player_id', 'fifa_version', 'short_name', 'long_name', 'player_positions', 
    'overall', 'potential', 'value_eur', 'value_dol', 'wage_eur', 'wage_dol', 
    'age', 'dob', 'height_cm', 'height_in', 'weight_kg', 'weight_lbs', 'club_team_id', 
    'club_name', 'league_id', 'league_name', 'league_level', 'club_position', 
    'club_jersey_number', 'club_loaned_from', 'club_joined_date', 'club_contract_valid_until_year', 
    'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 
    'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'work_rate', 
    'body_type', 'release_clause_eur', 'release_clause_dol', 'player_tags', 'player_traits'
]
player_info_df = df[player_info_columns]

# Creating player_attributes df
player_attributes_columns = [
    'player_id', 'fifa_version', 'short_name', 'long_name', 'player_positions', 'overall', 'pace', 
    'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 
    'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 
    'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 
    'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 
    'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 
    'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 
    'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 
    'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 
    'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle', 
    'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 
    'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed'
]
player_attributes_df = df[player_attributes_columns]

# Creating player_alternate_positions df
player_alternate_positions_columns = [
    'player_id', 'fifa_version', 'short_name', 'long_name', 'player_positions', 'overall', 'ls', 
    'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 
    'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 
    'rcb', 'rb', 'gk'
]
player_alternate_positions_df = df[player_alternate_positions_columns]

In [40]:
# Test if it worked

player_info_df.head()
player_attributes_df.head()
player_alternate_positions_df.head()

,player_id,short_name,long_name,player_positions,overall,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,90+3,90+3,90+3,91,91,91,91,91,89+3,89+3,89+3,89+3,81+3,81+3,81+3,89+3,68+3,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,E. Haaland,Erling Braut Haaland,ST,91,90+3,90+3,90+3,82,86,86,86,82,82+3,82+3,82+3,79+3,74+3,74+3,74+3,79+3,62+3,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,83+3,83+3,83+3,87,88,88,88,87,89+2,89+2,89+2,88+3,90+1,90+1,90+1,88+3,79+3,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,85+3,85+3,85+3,90,89,89,89,90,91-1,91-1,91-1,89+1,85+3,85+3,85+3,89+1,64+3,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,K. Benzema,Karim Benzema,"CF, ST",90,88+2,88+2,88+2,86,89,89,89,86,88+2,88+2,88+2,86+3,82+3,82+3,82+3,86+3,64+3,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3


In [47]:
# Create table queires
player_info_query = """
CREATE TABLE player_info (
    player_id INT,
    fifa_version INT,
    short_name VARCHAR(255),
    long_name VARCHAR(255),
    player_positions VARCHAR(255),
    overall INT,
    potential INT,
    value_eur INT,
    value_dol INT,
    wage_eur INT,
    wage_dol INT,
    age INT,
    dob DATE,
    height_cm INT,
    height_in FLOAT,
    weight_kg INT,
    weight_lbs FLOAT,
    club_team_id INT,
    club_name VARCHAR(255),
    league_id INT,
    league_name VARCHAR(255),
    league_level INT,
    club_position VARCHAR(255),
    club_jersey_number INT,
    club_loaned_from VARCHAR(255),
    club_joined_date DATE,
    club_contract_valid_until_year DATE,
    nationality_id INT,
    nationality_name VARCHAR(255),
    nation_team_id INT,
    nation_position VARCHAR(255),
    nation_jersey_number INT,
    preferred_foot VARCHAR(255),
    weak_foot INT,
    skill_moves INT,
    work_rate VARCHAR(255),
    body_type VARCHAR(255),
    release_clause_eur INT,
    release_clause_dol FLOAT,
    player_tags VARCHAR(255),
    player_traits VARCHAR(255),
    PRIMARY KEY (player_id, fifa_version)
);
"""

player_attributes_query = """
CREATE TABLE player_attributes (
    player_id INT,
    fifa_version INT,
    short_name VARCHAR(255),
    long_name VARCHAR(255),
    player_positions VARCHAR(255),
    overall INT,
    pace INT,
    shooting INT,
    passing INT,
    dribbling INT,
    defending INT,
    physic INT,
    attacking_crossing INT,
    attacking_finishing INT,
    attacking_heading_accuracy INT,
    attacking_short_passing INT,
    attacking_volleys INT,
    skill_dribbling INT,
    skill_curve INT,
    skill_fk_accuracy INT,
    skill_long_passing INT,
    skill_ball_control INT,
    movement_acceleration INT,
    movement_sprint_speed INT,
    movement_agility INT,
    movement_reactions INT,
    movement_balance INT,
    power_shot_power INT,
    power_jumping INT,
    power_stamina INT,
    power_strength INT,
    power_long_shots INT,
    mentality_aggression INT,
    mentality_interceptions INT,
    mentality_positioning INT,
    mentality_vision INT,
    mentality_penalties INT,
    mentality_composure INT,
    defending_marking_awareness INT,
    defending_standing_tackle INT,
    defending_sliding_tackle INT,
    goalkeeping_diving INT,
    goalkeeping_handling INT,
    goalkeeping_kicking INT,
    goalkeeping_positioning INT,
    goalkeeping_reflexes INT,
    goalkeeping_speed INT,
    PRIMARY KEY (player_id, fifa_version)
);
"""

player_alternate_positions_query = """
CREATE TABLE player_alternate_positions (
    player_id INT,
    fifa_version INT,
    short_name VARCHAR(255),
    long_name VARCHAR(255),
    player_positions VARCHAR(255),
    overall INT,
    ls VARCHAR(5),
    st VARCHAR(5),
    rs VARCHAR(5),
    lw VARCHAR(5),
    lf VARCHAR(5),
    cf VARCHAR(5),
    rf VARCHAR(5),
    rw VARCHAR(5),
    lam VARCHAR(5),
    cam VARCHAR(5),
    ram VARCHAR(5),
    lm VARCHAR(5),
    lcm VARCHAR(5),
    cm VARCHAR(5),
    rcm VARCHAR(5),
    rm VARCHAR(5),
    lwb VARCHAR(5),
    ldm VARCHAR(5),
    cdm VARCHAR(5),
    rdm VARCHAR(5),
    rwb VARCHAR(5),
    lb VARCHAR(5),
    lcb VARCHAR(5),
    cb VARCHAR(5),
    rcb VARCHAR(5),
    rb VARCHAR(5),
    gk VARCHAR(5),
    PRIMARY KEY (player_id, fifa_version)
);
"""

player_data_query = """
CREATE TABLE player_data (
    player_id INT,
    fifa_version INT,
    short_name VARCHAR(255),
    long_name VARCHAR(255),
    player_positions VARCHAR(255),
    overall INT,
    potential INT,
    value_eur INT,
    value_dol INT,
    wage_eur INT,
    wage_dol INT,
    age INT,
    dob DATE,
    height_cm INT,
    height_in FLOAT,
    weight_kg INT,
    weight_lbs FLOAT,
    club_team_id INT,
    club_name VARCHAR(255),
    league_id INT,
    league_name VARCHAR(255),
    league_level INT,
    club_position VARCHAR(255),
    club_jersey_number INT,
    club_loaned_from VARCHAR(255),
    club_joined_date DATE,
    club_contract_valid_until_year DATE,
    nationality_id INT,
    nationality_name VARCHAR(255),
    nation_team_id INT,
    nation_position VARCHAR(255),
    nation_jersey_number INT,
    preferred_foot VARCHAR(255),
    weak_foot INT,
    skill_moves INT,
    work_rate VARCHAR(255),
    body_type VARCHAR(255),
    release_clause_eur INT,
    release_clause_dol FLOAT,
    player_tags VARCHAR(255),
    player_traits VARCHAR(255),
    pace INT,
    shooting INT,
    passing INT,
    dribbling INT,
    defending INT,
    physic INT,
    attacking_crossing INT,
    attacking_finishing INT,
    attacking_heading_accuracy INT,
    attacking_short_passing INT,
    attacking_volleys INT,
    skill_dribbling INT,
    skill_curve INT,
    skill_fk_accuracy INT,
    skill_long_passing INT,
    skill_ball_control INT,
    movement_acceleration INT,
    movement_sprint_speed INT,
    movement_agility INT,
    movement_reactions INT,
    movement_balance INT,
    power_shot_power INT,
    power_jumping INT,
    power_stamina INT,
    power_strength INT,
    power_long_shots INT,
    mentality_aggression INT,
    mentality_interceptions INT,
    mentality_positioning INT,
    mentality_vision INT,
    mentality_penalties INT,
    mentality_composure INT,
    defending_marking_awareness INT,
    defending_standing_tackle INT,
    defending_sliding_tackle INT,
    goalkeeping_diving INT,
    goalkeeping_handling INT,
    goalkeeping_kicking INT,
    goalkeeping_positioning INT,
    goalkeeping_reflexes INT,
    goalkeeping_speed INT,
    ls VARCHAR(5),
    st VARCHAR(5),
    rs VARCHAR(5),
    lw VARCHAR(5),
    lf VARCHAR(5),
    cf VARCHAR(5),
    rf VARCHAR(5),
    rw VARCHAR(5),
    lam VARCHAR(5),
    cam VARCHAR(5),
    ram VARCHAR(5),
    lm VARCHAR(5),
    lcm VARCHAR(5),
    cm VARCHAR(5),
    rcm VARCHAR(5),
    rm VARCHAR(5),
    lwb VARCHAR(5),
    ldm VARCHAR(5),
    cdm VARCHAR(5),
    rdm VARCHAR(5),
    rwb VARCHAR(5),
    lb VARCHAR(5),
    lcb VARCHAR(5),
    cb VARCHAR(5),
    rcb VARCHAR(5),
    rb VARCHAR(5),
    gk VARCHAR(5),
    PRIMARY KEY (player_id, fifa_version)
);
"""

In [51]:
# Execute the create table statements

# player_info_query table
cursor.execute(player_info_query)
# player_attributes_query table
cursor.execute(player_attributes_query)
# player_alternate_positions_query table
cursor.execute(player_alternate_positions_query)
# player_data_query table
cursor.execute(player_data_query)

0

Define and list your SQL data types for your first table: <br><br>
**My SQL data types for my first table, player_info:**<br>

    player_id INT,
    fifa_version INT,
    short_name VARCHAR(255),
    long_name VARCHAR(255),
    player_positions VARCHAR(255),
    overall INT,
    potential INT,
    value_eur INT,
    value_dol INT,
    wage_eur INT,
    wage_dol INT,
    age INT,
    dob DATE,
    height_cm INT,
    height_in FLOAT,
    weight_kg INT,
    weight_lbs FLOAT,
    club_team_id INT,
    club_name VARCHAR(255),
    league_id INT,
    league_name VARCHAR(255),
    league_level INT,
    club_position VARCHAR(255),
    club_jersey_number INT,
    club_loaned_from VARCHAR(255),
    club_joined_date DATE,
    club_contract_valid_until_year DATE,
    nationality_id INT,
    nationality_name VARCHAR(255),
    nation_team_id INT,
    nation_position VARCHAR(255),
    nation_jersey_number INT,
    preferred_foot VARCHAR(255),
    weak_foot INT,
    skill_moves INT,
    work_rate VARCHAR(255),
    body_type VARCHAR(255),
    release_clause_eur INT,
    release_clause_dol FLOAT,
    player_tags VARCHAR(255),
    player_traits VARCHAR(255)

Why did you choose these values to make up your first database table? <br> <br>
I chose these values since out of the whole dataset, these are what coorelates with the base info about the player. <br> <br> <br>
What did you choose for your primary key and why? <br> <br>
I chose player_id and fifa_version as the primary key because the player_id signifies the unique id of the player where as the fifa_version tells us what version of the game the player_id is referencing. Both of them combined gives a single record.

In [54]:
# Print Columns
table_name = 'player_data'
query = f"SHOW COLUMNS FROM {table_name}"

# This only gives the count
cursor.execute(query)

# Get all column names
columns = cursor.fetchall()  

# Print column names
for column in columns:
    print(column[0])

player_id
fifa_version
short_name
long_name
player_positions
overall
potential
value_eur
value_dol
wage_eur
wage_dol
age
dob
height_cm
height_in
weight_kg
weight_lbs
club_team_id
club_name
league_id
league_name
league_level
club_position
club_jersey_number
club_loaned_from
club_joined_date
club_contract_valid_until_year
nationality_id
nationality_name
nation_team_id
nation_position
nation_jersey_number
preferred_foot
weak_foot
skill_moves
work_rate
body_type
release_clause_eur
release_clause_dol
player_tags
player_traits
pace
shooting
passing
dribbling
defending
physic
attacking_crossing
attacking_finishing
attacking_heading_accuracy
attacking_short_passing
attacking_volleys
skill_dribbling
skill_curve
skill_fk_accuracy
skill_long_passing
skill_ball_control
movement_acceleration
movement_sprint_speed
movement_agility
movement_reactions
movement_balance
power_shot_power
power_jumping
power_stamina
power_strength
power_long_shots
mentality_aggression
mentality_interceptions
mentality_pos

### Load the Tables with Data ###

In [78]:
# Connect with SQL Alchemy
password = 'chowdhury626'
username = 'root'
port = 3306
database_name = 'male_fifa_players'
host = 'localhost'

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database_name}')


/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl00000gn/T/ipykernel_91940/3957069423.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_info_df['club_joined_date'] = pd.to_datetime(player_info_df['club_joined_date'], errors='coerce')
/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl00000gn/T/ipykernel_91940/3957069423.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_info_df['club_joined_date'] = player_info_df['club_joined_date'].fillna(pd.Timestamp('1970-01-01'))
/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl000

In [ ]:
# Load player_info data

# I was getting errors that club_joined_date still had nulls and could not load to MySQL
    # Convert date columns to datetime format
        # Coerce code makes it so it becomes NaT
player_info_df['club_joined_date'] = pd.to_datetime(player_info_df['club_joined_date'], errors='coerce')

    # Any NaT will be replaced with placeholder 1970-01-01
player_info_df['club_joined_date'] = player_info_df['club_joined_date'].fillna(pd.Timestamp('1970-01-01'))

    # Convert back to string and make it yyyy-mm-dd format
player_info_df['club_joined_date'] = player_info_df['club_joined_date'].dt.strftime('%Y-%m-%d')

    # Load the data
player_info_df.to_sql(name='player_info', con=engine, if_exists='append', index=False)

In [83]:
# Load player_attributes data
# Fill missing fifa_version values with a 0
player_attributes_df['fifa_version'] = player_attributes_df['fifa_version'].fillna(0)

player_attributes_df.to_sql(name='player_attributes', con=engine, if_exists='append', index=False)

/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl00000gn/T/ipykernel_91940/1286636235.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_attributes_df['fifa_version'] = player_attributes_df['fifa_version'].fillna(0)


In [84]:
# Load player_alternate_positions data
player_alternate_positions_df.to_sql(name='player_alternate_positions', con=engine, if_exists='append', index=False)

In [86]:
# Load player_date data
# I was getting errors that club_joined_date still had nulls and could not load to MySQL
    # Convert date columns to datetime format
        # Coerce code makes it so it becomes NaT
df['club_joined_date'] = pd.to_datetime(df['club_joined_date'], errors='coerce')

    # Any NaT will be replaced with placeholder 1970-01-01
df['club_joined_date'] = df['club_joined_date'].fillna(pd.Timestamp('1970-01-01'))

    # Convert back to string and make it yyyy-mm-dd format
df['club_joined_date'] = df['club_joined_date'].dt.strftime('%Y-%m-%d')

df.to_sql(name='player_data', con=engine, if_exists='append', index=False)

In [106]:
# Test Queries
table_name = 'player_info'
query = f"SELECT * FROM {table_name}"

results = pd.read_sql(query, con)

display(results.head())

,player_id,fifa_version,short_name,long_name,player_positions,overall,potential,value_eur,value_dol,wage_eur,wage_dol,age,dob,height_cm,height_in,weight_kg,weight_lbs,club_team_id,club_name,league_id,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,work_rate,body_type,release_clause_eur,release_clause_dol,player_tags,player_traits
0,2,15,G. Pasquale,Giovanni Pasquale,LM,68,68,675000,742500,9000,9900,32,1982-01-05,181,71.26,82,180.78,55,Udinese,31,Serie A,1,LM,26,NULL,2008-07-01,1970-01-01,27,Italy,0,NULL,0,Left,3,2,Medium/Medium,Normal (170-185),0,0.0,NULL,NULL
1,2,16,G. Pasquale,Giovanni Pasquale,LM,69,69,625000,687500,7000,7700,33,1982-01-05,182,71.65,72,158.73,55,Udinese,31,Serie A,1,SUB,26,NULL,2008-07-01,1970-01-01,27,Italy,0,NULL,0,Left,3,2,Medium/High,Normal (170-185),0,0.0,NULL,NULL
2,11,15,R. Rocchi,Romain Rocchi,"CM, CDM",68,68,675000,742500,8000,8800,32,1981-10-02,183,72.05,75,165.35,68,Metz,16,Ligue 1,1,RDM,7,NULL,2013-07-01,1970-01-01,18,France,0,NULL,0,Right,3,3,Medium/High,Normal (170-185),0,0.0,NULL,Finesse Shot
3,16,17,Luis García,Luis García Fernández,"CM, CAM, ST, RM",70,70,450000,495000,8000,8800,35,1981-02-06,177,69.69,69,152.12,2013,AS Eupen,4,Jupiler Pro League,1,LCM,10,NULL,2014-07-19,1970-01-01,45,Spain,0,NULL,0,Right,3,3,Medium/Medium,Lean (170-185),0,0.0,NULL,Playmaker (AI)
4,16,18,Luis García,Luis García Fernández,"CM, CAM",70,70,575000,632500,8000,8800,36,1981-02-06,178,70.08,65,143.30,2013,AS Eupen,4,Jupiler Pro League,1,RCM,10,NULL,2014-07-19,1970-01-01,45,Spain,0,NULL,0,Right,3,3,Medium/Medium,Lean (170-185),834000,917400.0,NULL,"Playmaker (AI), Takes Finesse Free Kicks"


**STOP**<br>
To create a new schema diagram for your new database (Even though it only has one table...it's good practice!)<br>
Open MySQL Workbench again<br>
Click Home<br>
Click the Models icon<br>
Click the > icon to the right of "Models"<br>
Choose “Create EER Model from Database” <br>
The Reverse Engineer Database Wizard starts and will walk you through your first database schema diagram.<br>
Save your model. <br>
You can now add relationships and or modify tables...but for this assignment, all we need is that first table. <br>

**Add a screen shot of your first schema diagram (The table) to your repository/Blackboard subission.**

In [ ]:
#Close the database connection :)
cursor.close()
con.close()

Now what if we wanted to explore the number of each species observed within each separate body of water together? We'll begin JOINing tables in our next assignment.

### Creating a .db file from scratch locally and explore with MySQL in Python.

In [110]:
# Create the base file
db_file_path = "/Users/udoychowdhury/Documents/DataScience/Soccer Data/male_fifa_players.db"
conn = sqlite3.connect(db_file_path)

# Create a cursor object
cur = conn.cursor()

In [111]:
# Execute the create table statements

# player_info_query table
cur.execute(player_info_query)
# player_attributes_query table
cur.execute(player_attributes_query)
# player_alternate_positions_query table
cur.execute(player_alternate_positions_query)
# player_data_query table
cur.execute(player_data_query)

In [112]:
# Commit changes
conn.commit()

### Load the tables

In [114]:
# Load player_info data

# I was getting errors that club_joined_date still had nulls and could not load to MySQL
    # Convert date columns to datetime format
        # Coerce code makes it so it becomes NaT
player_info_df['club_joined_date'] = pd.to_datetime(player_info_df['club_joined_date'], errors='coerce')

    # Any NaT will be replaced with placeholder 1970-01-01
player_info_df['club_joined_date'] = player_info_df['club_joined_date'].fillna(pd.Timestamp('1970-01-01'))

    # Convert back to string and make it yyyy-mm-dd format
player_info_df['club_joined_date'] = player_info_df['club_joined_date'].dt.strftime('%Y-%m-%d')

    # Load the data
player_info_df.to_sql(name='player_info', con=conn, if_exists='replace', index=False)

/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl00000gn/T/ipykernel_91940/3444446565.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_info_df['club_joined_date'] = pd.to_datetime(player_info_df['club_joined_date'], errors='coerce')
/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl00000gn/T/ipykernel_91940/3444446565.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_info_df['club_joined_date'] = player_info_df['club_joined_date'].fillna(pd.Timestamp('1970-01-01'))
/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl00000

In [115]:
# Load player_attributes data
# Fill missing fifa_version values with a 0
player_attributes_df['fifa_version'] = player_attributes_df['fifa_version'].fillna(0)

player_attributes_df.to_sql(name='player_attributes', con=conn, if_exists='replace', index=False)

/var/folders/wm/z1f6_f9d3zb9gt8g3wkr4cl00000gn/T/ipykernel_91940/665349999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_attributes_df['fifa_version'] = player_attributes_df['fifa_version'].fillna(0)


In [116]:
# Load player_alternate_positions data
player_alternate_positions_df.to_sql(name='player_alternate_positions', con=conn, if_exists='replace', index=False)

In [120]:
# Load player_date data
# I was getting errors that club_joined_date still had nulls and could not load to MySQL
    # Convert date columns to datetime format
        # Coerce code makes it so it becomes NaT
df['club_joined_date'] = pd.to_datetime(df['club_joined_date'], errors='coerce')

    # Any NaT will be replaced with placeholder 1970-01-01
df['club_joined_date'] = df['club_joined_date'].fillna(pd.Timestamp('1970-01-01'))

    # Convert back to string and make it yyyy-mm-dd format
df['club_joined_date'] = df['club_joined_date'].dt.strftime('%Y-%m-%d')

df.to_sql(name='player_data', con=conn, if_exists='replace', index=False)

### Check if it worked

In [5]:
db_file_path = "/Users/udoychowdhury/Documents/DataScience/Soccer Data/male_fifa_players.db"
table_name = "player_info"

# Connect
conn = sqlite3.connect(db_file_path)
query = f"SELECT * FROM {table_name}"

# Execute query
df = pd.read_sql_query(query, conn)
display(df.head())

,player_id,fifa_version,short_name,long_name,player_positions,overall,potential,value_eur,value_dol,wage_eur,wage_dol,age,dob,height_cm,height_in,weight_kg,weight_lbs,club_team_id,club_name,league_id,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,work_rate,body_type,release_clause_eur,release_clause_dol,player_tags,player_traits
0,231747,24,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,181500000,199650000,230000,253000,24,1998-12-20,182,71.65,75,165.35,73,Paris Saint Germain,16,Ligue 1,1,LW,7,NULL,2018-07-01,1970-01-01,18,France,1335,LW,10,Right,4,5,High/Low,Unique,349400000,384340000,"#Speedster, #Dribbler, #Acrobat, #Clinical fin...","Quick Step +, Rapid, Flair, Trivela"
1,239085,24,E. Haaland,Erling Braut Haaland,ST,91,94,185000000,203500000,340000,374000,22,2000-07-21,195,76.77,94,207.23,10,Manchester City,13,Premier League,1,ST,9,NULL,2022-07-01,1970-01-01,36,Norway,1352,ST,9,Left,3,3,High/Medium,Unique,356100000,391710000,"#Aerial threat, #Distance shooter, #Strength, ...","Acrobatic +, Power Header, Quick Step"
2,192985,24,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,103000000,113300000,350000,385000,32,1991-06-28,181,71.26,75,165.35,10,Manchester City,13,Premier League,1,SUB,17,NULL,2015-08-30,1970-01-01,7,Belgium,1325,CAM,7,Right,5,4,High/Medium,Unique,190600000,209660000,"#Dribbler, #Playmaker, #Distance shooter, #Cro...","Pinged Pass +, Dead Ball, Incisive Pass, Long ..."
3,158023,24,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,41000000,45100000,23000,25300,36,1987-06-24,169,66.54,67,147.71,112893,Inter Miami,39,Major League Soccer,1,RF,10,NULL,2023-07-16,1970-01-01,52,Argentina,1369,RW,10,Left,4,4,Low/Low,Unique,61500000,67650000,"#Dribbler, #Playmaker, #FK Specialist, #Acroba...","Technical +, Finesse Shot, Dead Ball, Pinged P..."
4,165153,24,K. Benzema,Karim Benzema,"CF, ST",90,90,51000000,56100000,95000,104500,35,1987-12-19,185,72.83,81,178.57,607,Al Ittihad,350,Pro League,1,RS,9,NULL,2023-07-01,1970-01-01,18,France,0,NULL,0,Right,4,4,Medium/Medium,Normal (170-185),81600000,89760000,"#Poacher, #Aerial threat, #Clinical finisher, ...","Finesse Shot +, Dead Ball, Pinged Pass, Tiki T..."


In [6]:
table_name = "player_attributes"

# Connect
conn = sqlite3.connect(db_file_path)
query = f"SELECT * FROM {table_name}"

# Execute query
df = pd.read_sql_query(query, conn)
display(df.head())

,player_id,fifa_version,short_name,long_name,player_positions,overall,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,231747,24,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,97,90,80,92,36,78,78,94,73,86,84,93,80,69,71,92,97,97,93,93,82,90,88,88,77,83,64,38,93,83,84,88,26,34,32,13,5,7,11,6,0.0
1,239085,24,E. Haaland,Erling Braut Haaland,ST,91,89,93,66,80,45,88,47,96,83,77,90,79,77,62,53,82,82,94,76,94,72,94,93,76,93,86,87,43,96,74,84,87,38,47,29,7,14,13,11,7,0.0
2,192985,24,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,72,88,94,87,65,78,95,85,55,94,83,86,92,83,94,92,72,72,74,92,78,92,72,88,74,92,75,66,88,95,83,88,66,70,53,15,13,5,10,13,0.0
3,158023,24,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,80,87,90,94,33,64,83,89,60,91,86,96,93,93,90,93,87,74,91,88,95,83,71,70,68,90,44,40,91,92,75,96,20,35,24,6,11,15,14,8,0.0
4,165153,24,K. Benzema,Karim Benzema,"CF, ST",90,79,88,83,87,39,78,75,91,90,89,88,87,82,73,76,91,78,79,77,92,72,87,85,82,82,81,63,39,92,90,85,90,43,24,18,13,11,5,5,7,0.0


In [126]:
table_name = "player_alternate_positions"

# Connect
conn = sqlite3.connect(db_file_path)
query = f"SELECT * FROM {table_name}"

# Execute query
df = pd.read_sql_query(query, conn)
display(df.head())

,player_id,fifa_version,short_name,long_name,player_positions,overall,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,24,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,90+3,90+3,90+3,91,91,91,91,91,89+3,89+3,89+3,89+3,81+3,81+3,81+3,89+3,68+3,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,24,E. Haaland,Erling Braut Haaland,ST,91,90+3,90+3,90+3,82,86,86,86,82,82+3,82+3,82+3,79+3,74+3,74+3,74+3,79+3,62+3,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,24,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,83+3,83+3,83+3,87,88,88,88,87,89+2,89+2,89+2,88+3,90+1,90+1,90+1,88+3,79+3,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,24,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,85+3,85+3,85+3,90,89,89,89,90,91-1,91-1,91-1,89+1,85+3,85+3,85+3,89+1,64+3,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,24,K. Benzema,Karim Benzema,"CF, ST",90,88+2,88+2,88+2,86,89,89,89,86,88+2,88+2,88+2,86+3,82+3,82+3,82+3,86+3,64+3,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3


**STOP HERE**<br>
Before moving on, it is **important** to understand the difference of what we have just completed. Using RSQLite, we have created a database LOCALLY. Notice we did not specify a user, host, or password! We can find this database as a file on our machine. The .db file is created in the same location or working directory you are currently in. If you did not specify a working directory, the .db file is created where this .ipynb is located. 

In [7]:
#Close the database connection :)
conn.commit()
conn.close()